In [ ]:
import re, sys, math
import glob
import random
import json
import os 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from collections import OrderedDict
import pandas as pd
from helper_utilities import *
from helper_plotutilities import *
from unet_model  import *
from unet_predict_sk import *

%matplotlib inline


8

In [ ]:
# RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
# image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
# #label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"

# for results_file in bd176 :
#     m_name = results_file
#     pred_file = RESULTS_PATH + results_file + 'z_predictions.npy'
#     weights_file = RESULTS_PATH + results_file + '.hdf5'
#     print ("weights file", weights_file)
#     print ("pred_file", pred_file)


#     mnet = predict_with_pretrained_weights(model_name=m_name, nGPU=0, model_file=weights_file, image_size = 176, \
#                                 test_image_file = image_file)
#     #mnet.save_model_info(mpath)
#     print ("Saving predictions")
#     np.save(pred_file, mnet.predictions)

In [ ]:
#explist256 = ['combined_1_3_256_epoch20', 'combined_1_3_256', 'combined_aug_1_3_256',\
# 'combined_drop_1_3_256','combined_aug_drop_1_3_256']


explist256 = [ 'combined_1_3_256', 'combined_aug_1_3_256', 'combined_drop_1_3_256','combined_aug_drop_1_3_256']

#image set that includes labels with zero contours in the training set
explist256_0 = ['combined_1_3_0_256', 'combined_aug_1_3_0_256', 'combined_1_3_0_256_drop','combined_1_3_0_256_dicecoef', 'combined_1_3_0_256_drop_dicecoef']

#explist176 = ['combined_1_3_176', 'combined_1_3_176_aug_dice', 'combined_1_3_176_aug_drop_dice']
explist176 = ['combined_1_3_176', 'combined_1_3_176_dice', 'combined_1_3_176_aug_dice', 'combined_1_3_176_drop_dice', 'combined_1_3_176_aug_drop_dice']

explist176_0 = ['1_3_0_176_bn','combined_1_3_0_176_dice', 'combined_1_3_0_176_aug_dice','combined_1_3_0_176_drop_dice', 'combined_1_3_0_176_aug_drop_dice', 'combined_1_3_0_176_aug_drop_dice2']


In [ ]:
bn176 = [ '1_3_0_176_bn' , '1_3_0_176_bn2', '1_3_0_176_bn4', '1_3_0_176_bn_aug_drop']

aug176 = [  '1_3_0_176_aug_dice', '1_3_0_176_aug2_dice1', '1_3_0_176_aug2_dice2', '1_3_0_176_aug2_dice3', \
    '1_3_0_176_aug_drop_dice', '1_3_0_176_aug_drop_dice2', '1_3_0_176_aug_bd', '1_3_0_176_aug_bd2', \
    '1_3_0_176_aug_ld1', '1_3_0_176_aug_ld2', '1_3_0_176_aug_ld3', '1_3_0_176_aug_drop_ld1', \
    '1_3_0_176_aug_drop_ld2', '1_3_0_176_aug_drop_ld3', '1_3_0_176_aug_bce', '1_3_0_176_aug_bce2'  \
    ]



In [ ]:
RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/experiments/'
label_file176_0 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy'
keys = get_perf_keys()
perf176bn = pd.DataFrame( keys, columns=["Attribute"] )
for results_file in bn176 :
    print (results_file)
    pred_file = RESULTS_PATH + results_file + '_predictions.npy'
    perf = compute_performance_statistics(label_file176_0, pred_file)
    perf176bn[results_file]= perf176bn['Attribute'].map(perf)

In [ ]:
perf176bn

In [ ]:

file_name = '/masvol/heartsmart/unet_model/baseline/results/experiments/1_3_0_176_perf.csv'
perf176_0_df.to_csv(file_name)

In [ ]:
RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/experiments/'
label_file176_0 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy'
keys = get_perf_keys()
perf176_0_df = pd.DataFrame( keys, columns=["Attribute"] )
for results_file in aug176 :
    print (results_file)
    pred_file = RESULTS_PATH + results_file + '_predictions.npy'
    perf = compute_performance_statistics(label_file176_0, pred_file)
    perf176_0_df[results_file]= perf176_0_df['Attribute'].map(perf)
    

In [ ]:
perf176_0_df

In [ ]:
# RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
# label_file176 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_labels.npy'

# perf176_df = pd.DataFrame( keys, columns=["Attribute"] )
# for results_file in explist176 :
#     pred_file = RESULTS_PATH + results_file + '_predictions.npy'
#     perf = compute_performance_statistics(label_file176, pred_file)
#     perf176_df[results_file]= perf176_df['Attribute'].map(perf)
    

In [ ]:
perf176_0_df[['Attribute','1_3_0_176_aug2_dice2',  '1_3_0_176_aug_bce' , '1_3_0_176_aug_bd2', '1_3_0_176_aug_drop_dice', '1_3_0_176_aug_ld1']]

In [ ]:
# RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
# label_file256 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_labels.npy'

# perf256_df = pd.DataFrame( keys, columns=["Attribute"] )
# for results_file in explist256 :
#     pred_file = RESULTS_PATH + results_file + '_predictions.npy'
#     perf = compute_performance_statistics(label_file256, pred_file)
#     perf256_df[results_file]= perf256_df['Attribute'].map(perf)

In [ ]:
# RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
# label_file256_0 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy'

# perf256_0_df = pd.DataFrame( keys, columns=["Attribute"] )
# for results_file in explist256_0 :
#     pred_file = RESULTS_PATH + results_file + '_predictions.npy'
#     perf = compute_performance_statistics(label_file256_0, pred_file)
#     perf256_0_df[results_file]= perf256_0_df['Attribute'].map(perf)

In [ ]:
perf256_df

In [ ]:
file_name = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_perf_comparison_old.csv'
perf176_0_df.to_csv(file_name, sep='\t')

In [ ]:
file_name = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_perf_comparison_old.csv'
perf176_df.to_csv(file_name, sep='\t')

In [ ]:
file_name = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_perf_comparison.csv'
perf256_0_df.to_csv(file_name, sep='\t')

In [ ]:
file_name = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_perf_comparison.csv'
perf256_df.to_csv(file_name, sep='\t')

In [ ]:
#hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_drop_dice_learning_history.json'
#hist = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn_learning_history.json'
hist = '/masvol/heartsmart/unet_model/baseline/results/experiments/combined_1_3_176x_aug_dice1_learning_history.json'

plot_learning_history(hist)

In [ ]:
#hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_dicecoef_learning_history.json'
#hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dicecoef_learning_history.json'
#hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_learning_history.json'
hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176x_aug_dice2_learning_history.json'
#hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_dice_learning_history.json'

plot_learning_history(hist)

In [ ]:
perf176_df

In [ ]:
hist = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice_learning_history.json'
#hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_dice_learning_history.json'

plot_learning_history(hist)

In [ ]:
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/experiments/1_3_0_176_aug2_dice1_predictions.npy"
y_true = np.load(label_file)
y_pred = np.load(pred_file)


y_true = y_true.flatten()
y_pred = y_pred.flatten()
smooth = 1.
intersection = np.sum(y_true * y_pred)
score = (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)


In [ ]:
score

In [ ]:
perf176_df

In [ ]:
for results_file in explist256 :
    history_file = RESULTS_PATH + results_file + '_learning_history.json'
    print ('='*30)
    print ('='*30)
    print ("Learning history of :" + results_file)
    print ('-'*30)
    
    plot_learning_history(history_file)
    

In [ ]:
for results_file in explist176 :
    history_file = RESULTS_PATH + results_file + '_learning_history.json'
    print ('='*30)
    print ('='*30)
    print ("Learning history of :" + results_file)
    print ('-'*30)
    
    plot_learning_history(history_file)

In [ ]:
# label_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_labels.npy"
# pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_predictions.npy"
# y_true_f = np.load(label_file)
# y_pred_f = np.load(pred_file)
# p = get_performance_statistics(y_true_f, y_pred_f)
# print (p)

In [ ]:
# perf_file = "perf_values.json"
# print ("Saving Performance values", perf_file)
# with open(perf_file, 'w') as file:
#     json.dump(p, file, indent=2)
# print('-'*30)

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dicep_predictions.npy"
#plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


## Analysis on training with 256x256  combined data set (ACDC + Sunnybrook)
### Below are the results of 4 tests
1. Training with combined dataset
2. Training with combined dataset + Augmentation
3. Training with combined dataset (no augmentation),  using Unet with dropout layers
4. Training with combined dataset + Augmentation,  using Unet with dropout layers

## Analysis on training with 256x256  combined data set (ACDC + Sunnybrook) (No labels with zero contours)

### data set prefix combined_1_3_256 

### Below are the results of 4 tests
1. Training with combined dataset
2. Training with combined dataset + Augmentation
3. Training with combined dataset (no augmentation),  using Unet with dropout layers
4. Training with combined dataset + Augmentation,  using Unet with dropout layers

In [ ]:
# zcontour = [50, 53, 72, 73, 116, 118, 145, 188]
# display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_aug_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_aug_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_aug_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_aug_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_aug_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_aug_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


## Analysis on training with 256x256  combined data set (ACDC + Sunnybrook) + Labels with Zero Contours  
### data set prefix combined_1_3_0_256 
### Below are the results of 4 tests
1. Training with combined dataset
2. Training with combined dataset + Augmentation
3. Training with combined dataset (no augmentation),  using Unet with dropout layers
4. Training with combined dataset + Augmentation,  using Unet with dropout layers

In [ ]:
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dicecoef_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dicecoef_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [21, 84, 127, 220]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_aug_dice.hdf5" 
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_aug_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_aug_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
zcontour = [2, 10, 13, 21, 38, 50, 72, 86, 87, 113, 116, 118, 151, 152, 184, 202, 203, 230]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_aug_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_aug_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_aug_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


## Analysis on training with 176x176  combined data set (ACDC + Sunnybrook)  ( no Labels with zero contours)
### (data set prefix combined_1_3_176)
### Below are the results of 4 tests
1. Training with combined dataset
2. Training with combined dataset + Augmentation
3. Training with combined dataset (no augmentation),  using Unet with dropout layers
4. Training with combined dataset + Augmentation,  using Unet with dropout layers

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/experiments/combined_1_3_176_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/experiments/combined_1_3_176x_aug_dice2_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/experiments/combined_1_3_176x_aug_dice2_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
# zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
# display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
zcontour = [55, 66, 81, 92, 117, 141, 173, 226]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/experiments/combined_1_3_176_aug_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/experiments/combined_1_3_176_aug_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
zcontour = [173, 188]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
zcontour = [55, 81, 82, 141]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

## Analysis on training with 176x176  combined data set (ACDC + Sunnybrook) + Labels with Zero Contours  
### (data set prefix combined_1_3_0_176)
### Below are the results of 4 tests
1. Training with combined dataset
2. Training with combined dataset + Augmentation
3. Training with combined dataset (no augmentation),  using Unet with dropout layers
4. Training with combined dataset + Augmentation,  using Unet with dropout layers

In [ ]:

weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
# zcontour = [8, 14, 20, 32, 39, 95, 135]
# display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
# Sample Index of labels with zero contours [8, 14, 20, 32, 39, 95, 135]
# Sample Index of predictions with zero contours [14, 20, 77, 95, 135, 169, 189]
# Sample Index where missed predictions >= 100% 6 [8, 32, 39, 77, 169, 189]
# zcontour = [8, 14, 20, 32, 39, 95, 135]
# display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
# zcontour = [14, 20, 77, 95, 135, 169, 189]
# display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
# zcontour = [8, 32, 39, 77, 169, 189]
# #display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)
# display_images_predictions (image_file,  pred_file, image_list = zcontour)

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [4, 8, 32, 39, 46, 63, 65, 77,]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)
#combined_1_3_0_176_drop_dice

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)
#combined_1_3_0_176_drop_dice

In [ ]:
# zcontour = [8, 14, 20, 32, 39, 95, 135]
# display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
# #zcontour = [14, 20, 46, 65, 77, 102, 104, 154, 169, 189]
# zcontour = [14, 20, 63, 77, 95, 104, 135, 169, 189, 198, 212]
# display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
## 

## 176x176 images.. Training with BatchNormalization

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [4, 8, 32, 39, 63, 65, 67, 77, 83, 104, 144, 169, 198, 201]
display_images_predictions (image_file, pred_file, image_list = zcontour)
#combined_1_3_0_176_drop_dice

In [ ]:
zcontour = [8, 32, 39, 77, 169, 189]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
#'1_3_0_176_bn_drop_learning_history.json'
weights_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn_drop.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn_drop_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn_drop_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [4, 8, 32, 39, 63, 65, 67, 77, 83, 104, 144, 169, 198, 201]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)
#combined_1_3_0_176_drop_dice

In [ ]:
#'1_3_0_176_bn_drop_learning_history.json'
weights_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn2.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn2_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn2_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [4, 8, 32, 39, 63, 65, 67, 77, 83, 104, 144, 169, 198, 201]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)
#combined_1_3_0_176_drop_dice

## Analysis of predictions on  DSB data set (256x256  images)

In [ ]:
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_images.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_predictions.npy"
find_outliers_in_prediction(pred_file)
display_images_predictions (image_file, pred_file, num_images = 4, random_images = True)
print ("-"*30)
imglist = [21, 84, 127]
display_images_predictions (image_file, pred_file, image_list = imglist)


## Analysis of predictions on  DSB data set (176x176  images)

In [ ]:
# image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_176_test_images.npy"
# pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_176_predictions.npy"
# find_outliers_in_prediction(pred_file)
# display_images_predictions (image_file, pred_file, num_images = 4, random_images = True)
# print ("-"*30)
# print ("-"*30)
# imglist = [173]
# display_images_predictions (image_file, pred_file, image_list = imglist)

In [ ]:
RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/experiments/'
#label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"


m_name = 'dsb_315_176'
image_file = "/masvol/output/dsb/norm/outlier_testing/m1t3/" + m_name+ "_train.npy"
pred_file = RESULTS_PATH + m_name + '_predictions.npy'
#weights_file = RESULTS_PATH + 'combined_1_3_176x_aug_dice2.hdf5'
weights_file = '/masvol/heartsmart/unet_model/baseline/results/final/1_3_0_176_aug_ld1.hdf5'
mnet = predict_with_pretrained_weights(model_name=m_name, nGPU=0, model_file=weights_file, image_size = 176, \
                            test_image_file = image_file)

#mnet.save_model_info(mpath)
print ("Saving predictions")
np.save(pred_file, mnet.predictions)
find_outliers_in_prediction(pred_file)

In [ ]:

display_images_predictions (image_file, pred_file, num_images = 10, random_images = True)
print ("-"*30)


In [ ]:
imglist = [25, 29, 38, 50, 52, 55, 58, 64, 115, 117, 119, 187, 199, 202, 203, 211, 214, 246, 247, 282, 311, 314, 324, 326, 329, 343, 370, 373, 374, 386, 389, 397, 408, 410]
display_images_predictions (image_file, pred_file, image_list = imglist)